In [97]:
import json
from openai import OpenAI
from elasticsearch import Elasticsearch
import requests
import tiktoken

In [ ]:
client = OpenAI()

In [98]:
es_client = Elasticsearch('http://localhost:9200')

In [99]:
#Question 1
es_client.info()

ObjectApiResponse({'name': 'd4c83cd0ac28', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'px0lcgQ3St6mr80ZWzhBAA', 'version': {'number': '8.17.6', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'dbcbbbd0bc4924cfeb28929dc05d82d662c527b7', 'build_date': '2025-04-30T14:07:12.231372970Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [100]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

In [ ]:
from tqdm.auto import tqdm

In [ ]:
#Question 2
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

In [ ]:
query = 'How do execute a command on a Kubernetes pod?'

In [ ]:
def elastic_search(query):

    search_query = {
        "size": 1,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }
    
    response = es_client.search(index=index_name, body=search_query)
    
    return response['hits']

In [ ]:
#Question 3
elastic_search(query)

In [101]:
query = 'How do copy a file to a Docker container?'

In [102]:
def elastic_search(query):

    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])  
    
    return result_docs

In [103]:
#Question 4
es_response = elastic_search(query)

In [77]:
context_template = """
Q: {question}
A: {text}
""".strip()


context = "\n\n".join([
    context_template.format(question=record["question"], text=record["text"])
    for record in es_response
])

print(context)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

In [81]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    context = "\n\n".join([
        context_template.format(question=record["question"], text=record["text"])
        for record in es_response
    ])
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [84]:
#Question 5
prompt = build_prompt(query, es_response)
len(prompt)

1447

In [85]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [88]:
tokens = encoding.encode(prompt)

In [90]:
#Question 6
print(len(tokens))

321


In [91]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [93]:
#Bonus Question 
answer = llm(prompt)

In [94]:
print(answer)

To copy a file from your local machine to a Docker container, you can use the `docker cp` command. The basic syntax is as follows:

```bash
docker cp /path/to/local/file_or_directory container_id:/path/in/container
``` 

Replace `/path/to/local/file_or_directory` with the path to the file or directory you want to copy from your local machine, and `container_id:/path/in/container` with the destination path in the running Docker container.
